In [ ]:
elastic_rods_dir = '../elastic_rods/python/'
weaving_dir = './'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, optimization_visualization_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
importlib.reload(optimization_visualization_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, contact_optimization, get_double_side_view, show_selected_joints, highlight_rod_and_joint, write_distance_to_linkage_mesh, get_max_distance_to_target_surface, set_design_parameters_from_topology, get_fixed_boundary_joint)
import vis.fields
import matplotlib.cm as cm
import time
from optimization_visualization_helper import (compute_visualization_data_from_raw_data, get_objective_components_stage1, get_objective_components_stage2, get_objective_components_stage3, set_figure_label_and_limit, Visualization_Setting, plot_objective, plot_ribbon_component_analysis, render_video, combine_three_stage_video)
import json

In [ ]:
with open(osp.join(weaving_dir + 'woven_model.json')) as f:
    data = json.load(f)

In [ ]:
benchmark_data = []
data_root = 'mega_monster_optimization_diagram_results/twelfth_round/'

number of crossings, number of segments, number of ribbons, number of simulation variable, number of design variable, 


validation:  number of physical ribbons, deviation to surface, deviation to scan, deviation from scan to surface

### Get stats from initialization.

In [ ]:
# for i, model_info in enumerate(data['models']):
#     # Load model info from the json data.
#     thickness, width, name, use_constant_width, width_scale, scale_joint_weight, update_attraction_weight, number_of_updates, fix_boundary, only_two_stage = model_info['thickness'], model_info['width'], model_info['name'], model_info['constant_cross_section'], model_info['cross_section_scale'], model_info['scale_joint_weight'], model_info['update_attraction_weight'], model_info['number_of_updates'], model_info['fix_boundary'], model_info['only_two_stage']
    
#     # Set the interleaving type with special consideration for the bunny.
#     interleaving_type = InterleavingType.triaxialWeave
#     if name in ['bunny_head_small_triaxial_1', 'owl_1', 'clam_1']:
#         interleaving_type = InterleavingType.weaving
#     io = InputOrganizer(name, thickness, width, weaving_dir)
#     print( io.SURFACE_PATH, io.MODEL_PATH,  io.RIBBON_CS,  io.SUBDIVISION_RESOLUTION, interleaving_type,  use_constant_width,  width_scale)
#     # Initialize the linkage class.
#     with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, interleaving_type=interleaving_type, use_constant_width = use_constant_width, width_scale = width_scale)
#     # Fix the twisted ribbon for the bunny.
#     if name == 'bunny_head_small_triaxial_1':
#         print('Recompute normal for bunny')
#         input_joint_normals = np.reshape(curved_linkage.get_closest_point_normal(curved_linkage.jointPositions()), (curved_linkage.numJoints(), 3))
#         current_joint_normals = [curved_linkage.joint(i).normal for i in range(curved_linkage.numJoints())]
#         current_joint_normals[35] = input_joint_normals[35]
#         with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, use_constant_width=use_constant_width, interleaving_type=InterleavingType.triaxialWeave, input_joint_normals = current_joint_normals)

#     # Set design parameter to include both rest length and rest curvature.
#     curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
#     ribbons = order_segments_by_ribbons(curved_linkage)
#     data['models'][i]['num_joints'] = curved_linkage.numJoints()
#     data['models'][i]['num_segments'] = curved_linkage.numSegments()
#     data['models'][i]['num_ribbons'] = len(ribbons)
#     data['models'][i]['num_sim_vars'] = curved_linkage.numDoF()
#     data['models'][i]['num_design_vars'] = curved_linkage.numExtendedDoFPSRL() - curved_linkage.numDoF()

# with open(osp.join(weaving_dir + 'woven_model.json'), 'w') as f:
#     json.dump(data, f, indent=4)

### Get Optimization Data

In [ ]:
import py_newton_optimizer

# Optimization parameters.
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1;
OPTS.beta = 1e-6
OPTS.niter = 2000
OPTS.verboseNonPosDef = False
rw = 0.1
sw = 10
drw = 0.001
dsw = 0.01

In [ ]:
import pickle, gzip
benchmark_data = []
for model_info in data['models']:
    thickness, width, name, use_constant_width, width_scale, scale_joint_weight, update_attraction_weight, number_of_updates, fix_boundary, only_two_stage, num_joints, num_segments, num_ribbons, num_sim_vars, num_design_vars = model_info['thickness'], model_info['width'], model_info['name'], model_info['constant_cross_section'], model_info['cross_section_scale'], model_info['scale_joint_weight'], model_info['update_attraction_weight'], model_info['number_of_updates'], model_info['fix_boundary'], model_info['only_two_stage'], model_info['num_joints'], model_info['num_segments'], model_info['num_ribbons'], model_info['num_sim_vars'], model_info['num_design_vars']
    interleaving_type = InterleavingType.triaxialWeave
    if name in ['bunny_head_small_triaxial_1', 'owl_1', 'clam_1']:
        interleaving_type = InterleavingType.weaving
    io = InputOrganizer(name, thickness, width, weaving_dir)
    stage_1_data_filename = '{}/{}_stage_1.npy'.format('{}/{}'.format(data_root, name), name)
    stage_2_data_filename = '{}/{}_stage_2.npy'.format('{}/{}'.format(data_root, name), name)
    stage_2_weight_change_filename = '{}/{}_stage_2_weight_change.npy'.format('{}/{}'.format(data_root, name), name)
    stage_3_data_filename = '{}/{}_stage_3.npy'.format('{}/{}'.format(data_root, name), name)
    stage_2_solver_status_filename = '{}/{}_stage_2_solverStatus.npy'.format('{}/{}'.format(data_root, name), name)
    stage_3_solver_status_filename = '{}/{}_stage_3_solverStatus.npy'.format('{}/{}'.format(data_root, name), name)

    def get_time_from_data(stage_data):
        return sum([pt['iteration_time'] for pt in stage_data])
    def get_time_string(sec, status = 0):
        if sec == 0:
            return 'N/A'
        m, s = divmod(sec, 60)
        h, m = divmod(m, 60)
        time_string = ''
        if h == 0:
            time_string = '{:02d}:{:02d}'.format(int(np.round(m)), int(np.round(s)))
        else:
            time_string = '{:d}:{:02d}:{:02d}'.format(int(np.round(h)), int(np.round(m)), int(np.round(s)))
        if status < -300:
            return '{} L'.format(time_string)
        elif status < -100: 
            return '{} X'.format(time_string)
        else:
            return time_string
        
    stage_1_data = np.load(stage_1_data_filename, allow_pickle = True)
    stage_2_data = np.load(stage_2_data_filename, allow_pickle = True)
    stage_1_time = get_time_from_data(stage_1_data)
    stage_2_time = get_time_from_data(stage_2_data)
    
    stage_2_solverStatus = np.load(stage_2_solver_status_filename, allow_pickle = True)
    stage_3_solverStatus = np.load(stage_3_solver_status_filename, allow_pickle = True) if os.path.isfile(stage_3_solver_status_filename) else 0
    
    if not only_two_stage:
        stage_3_data = np.load(stage_3_data_filename, allow_pickle = True)
        stage_3_time = get_time_from_data(stage_3_data)
    else:
        stage_3_time = 0
    total_time = stage_1_time + stage_2_time + stage_3_time
    
    stage_2_status = 0
    stage_3_status = 0
    if np.sum(stage_2_solverStatus) != 0:
        print(name, 'stage 2 solver: ', stage_2_solverStatus)
        stage_2_status = np.min(stage_2_solverStatus)
    if np.sum(stage_3_solverStatus) != 0:
        print(name, 'stage 3 solver: ', stage_3_solverStatus)
        stage_3_status = np.min(stage_3_solverStatus)

        
#     distance_to_target = 0
#     if only_two_stage:
#         curved_linkage = pickle.load(gzip.open('{}/{}/{}_stage_2.pkl.gz'.format(data_root, name, name), 'r'))
#         distance_to_target = np.round(get_max_distance_to_target_surface(curved_linkage) / curved_linkage.get_l0(), 4)
#         from structural_analysis import weavingCrossingForceMagnitudes
#         cfm = weavingCrossingForceMagnitudes(curved_linkage, True)
#         normalActivationThreshold = np.percentile(cfm[:, 0], 75)
#         print(name, "Normal Activation Threshold: ", normalActivationThreshold)
#         print(name, "Potential activation threshold: ", -np.percentile(np.abs(cfm[:, 0]), 25))
#     else:
#         curved_linkage = pickle.load(gzip.open('{}/{}/{}_stage_3.pkl.gz'.format(data_root, name, name), 'r'))
#         distance_to_target = np.round(get_max_distance_to_target_surface(curved_linkage) / curved_linkage.get_l0(), 4)
#         curved_linkage = pickle.load(gzip.open('{}/{}/{}_stage_2.pkl.gz'.format(data_root, name, name), 'r'))
#         from structural_analysis import weavingCrossingForceMagnitudes
#         cfm = weavingCrossingForceMagnitudes(curved_linkage, True)
#         normalActivationThreshold = np.percentile(cfm[:, 0], 75)
#         print(name, "Normal Activation Threshold: ", normalActivationThreshold)
#         print(name, "Potential activation threshold: ", -np.percentile(np.abs(cfm[:, 0]), 25))
#     curved_linkage = pickle.load(gzip.open('{}/{}/{}_stage_1.pkl.gz'.format(data_root, name, name), 'r'))
#     curved_linkage.attraction_weight = 1e-5
#     with so(): elastic_rods.compute_equilibrium(curved_linkage, callback = None, options = OPTS, fixedVars = [])
#     distance_to_target_stage_1 = np.round(get_max_distance_to_target_surface(curved_linkage) / curved_linkage.get_l0(), 4)
    # Initialize the linkage class.
#     start_time = time.time()
#     with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, interleaving_type=interleaving_type, use_constant_width = use_constant_width, width_scale = width_scale)
#     print( io.SURFACE_PATH, io.MODEL_PATH,  io.RIBBON_CS,  io.SUBDIVISION_RESOLUTION, interleaving_type,  use_constant_width,  width_scale)
#     # Fix the twisted ribbon for the bunny.
#     if name == 'bunny_head_small_triaxial_1':
#         print('Recompute normal for bunny')
#         input_joint_normals = np.reshape(curved_linkage.get_closest_point_normal(curved_linkage.jointPositions()), (curved_linkage.numJoints(), 3))
#         current_joint_normals = [curved_linkage.joint(i).normal for i in range(curved_linkage.numJoints())]
#         current_joint_normals[35] = input_joint_normals[35]
#         with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, use_constant_width=use_constant_width, interleaving_type=InterleavingType.triaxialWeave, input_joint_normals = current_joint_normals)
#     set_design_parameters_from_topology(curved_linkage, io)
#     curved_linkage.attraction_weight = 1e-5
#     fixed_boundary_joints = []
#     if fix_boundary:
#         fixed_boundary_joints = get_fixed_boundary_joint(curved_linkage)
#     with so(): elastic_rods.compute_equilibrium(curved_linkage, callback = None, options = OPTS, fixedVars = fixed_boundary_joints)
#     distance_to_target_initialization = np.round(get_max_distance_to_target_surface(curved_linkage) / curved_linkage.get_l0(), 4)

    num_physical_ribbons = 'TBD'
    benchmark_data.append([model_info['display_name'], num_joints, num_segments, num_ribbons, num_physical_ribbons, num_sim_vars, num_design_vars, get_time_string(stage_1_time), get_time_string(stage_2_time, stage_2_status), get_time_string(stage_3_time, stage_3_status), get_time_string(total_time), 'TBD', 'TBD', 'TBD', 'TBD'])
with open(osp.join(weaving_dir + 'benchmark_data.json'), 'w') as f:
    json.dump(benchmark_data, f, indent=4)

In [ ]:
import plotly.graph_objects as go

headers = ['Model Name', 'Number of Crossings', 'Number of Ribbon Segments', 'Number of Ribbons', 'Number of Physical Ribbons', 'Equilibrium DoFs', 'Design DoFs', 'Stage 1 Time', 'Stage 2 Time', 'Stage 3 Time', 'Total Time', 'Norm. Max. Dist. Simulation to Target Surface', 'Norm. Max. Dist. Scan to Target Surface']
headers = ['Model Name', '# Crs.', '# Segments', '# Rbn.', '# Physical Rbn.', 'len(x)', 'len(p)', 'Stage I', 'Stage II', 'Stage III', 'Total Time', 'Sim. Fit', 'Scan Fit', 'Stage 1 Fit', 'Init. Fit']
# headers = ['Model Name', 'Number of Crossings', 'Number of Segments', 'DoF', 'Initialization Time (s)', 'Stage 1 Time (s)', 'Stage 2 Time (s)']

values = np.transpose(benchmark_data)
values = np.delete(values, 2, 0)
headers = np.delete(headers, 2, 0)
# values = np.array([weaving_display_name, weaving_num_crossings, weaving_num_segments, weaving_dof, weaving_initialization, weaving_stage_1, weaving_stage_2])

values = values[:, values[0].argsort()]
values = values[:, values[1].astype(int).argsort()]
# values = values[:, weaving_num_crossings.argsort()]
fig = go.Figure(data=[go.Table(header=dict(values=headers),
                 cells=dict(values=values, align = 'center'))
                     ])
fig.update_layout(height = 1000)
fig.show()

In [ ]:
values[10]

In [ ]:
headers

In [ ]:
# import pandas as pd
# df = pd.DataFrame(np.transpose(values))
# with open('../../Paper/CurWe/benchmark_table.tex','w') as tf:
#     tf.write(df.to_latex(header = list(headers), index = False))